In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
Read datasets
fake = pd.read_csv("data/Fake.csv")
true = pd.read_csv("data/True.csv")
fake.shape
(23481, 4)
true.shape
(21417, 4)
Data cleaning and preparation
# Add flag to track fake and real
fake['target'] = 'fake'
true['target'] = 'true'
# Concatenate dataframes
data = pd.concat([fake, true]).reset_index(drop = True)
data.shape
(44898, 5)
# Shuffle the data
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
# Check the data
data.head()
title	text	subject	date	target
0	Portuguese ex-PM Socrates indicted on corrupti...	LISBON (Reuters) - Former Portuguese prime min...	worldnews	October 11, 2017	true
1	Boiler Room EP #113 – ‘CNN is ISIS’	Tune in to the Alternate Current Radio Network...	Middle-east	June 16, 2017	fake
2	HILLARY GOT DESTROYED By Chris Wallace On FOX ...	Hillary shouldn t be on FOX News giving interv...	left-news	Aug 3, 2016	fake
3	Trump recommits to U.S. allies but says they m...	WASHINGTON (Reuters) - President Donald Trump ...	politicsNews	March 1, 2017	true
4	OAS says Honduran vote results in doubt due to...	TEGUCIGALPA (Reuters) - Observers cannot be ce...	worldnews	December 4, 2017	true
# Removing the date (we won't use it for the analysis)
data.drop(["date"],axis=1,inplace=True)
data.head()
title	text	subject	target
0	Trump uses policy speech to attack media, prom...	GETTSYBURG, Pa. (Reuters) - U.S. Republican pr...	politicsNews	true
1	Liz Cheney's Wyoming campaign backed by big na...	CODY, Wyo. (Reuters) - Former Vice President D...	politicsNews	true
2	Togolese to vote on presidential term limits a...	LOME (Reuters) - A bill to limit presidents in...	worldnews	true
3	Hillary Clinton says U.S. threats of war with ...	SEOUL (Reuters) - Former U.S. presidential can...	politicsNews	true
4	Trump administration, world financial official...	WASHINGTON (Reuters) - The Trump administratio...	politicsNews	true
# Removing the title (we will only use the text)
data.drop(["title"],axis=1,inplace=True)
data.head()
text	subject	target
0	GETTSYBURG, Pa. (Reuters) - U.S. Republican pr...	politicsNews	true
1	CODY, Wyo. (Reuters) - Former Vice President D...	politicsNews	true
2	LOME (Reuters) - A bill to limit presidents in...	worldnews	true
3	SEOUL (Reuters) - Former U.S. presidential can...	politicsNews	true
4	WASHINGTON (Reuters) - The Trump administratio...	politicsNews	true
# Convert to lowercase

data['text'] = data['text'].apply(lambda x: x.lower())
data.head()
text	subject	target
0	gettsyburg, pa. (reuters) - u.s. republican pr...	politicsNews	true
1	cody, wyo. (reuters) - former vice president d...	politicsNews	true
2	lome (reuters) - a bill to limit presidents in...	worldnews	true
3	seoul (reuters) - former u.s. presidential can...	politicsNews	true
4	washington (reuters) - the trump administratio...	politicsNews	true
# Remove punctuation

import string

def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

data['text'] = data['text'].apply(punctuation_removal)
# Check
data.head()
text	subject	target
0	gettsyburg pa reuters us republican president...	politicsNews	true
1	cody wyo reuters former vice president dick c...	politicsNews	true
2	lome reuters a bill to limit presidents in to...	worldnews	true
3	seoul reuters former us presidential candidat...	politicsNews	true
4	washington reuters the trump administration h...	politicsNews	true
# Removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/faviovazquez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
data.head()
text	subject	target
0	gettsyburg pa reuters us republican presidenti...	politicsNews	true
1	cody wyo reuters former vice president dick ch...	politicsNews	true
2	lome reuters bill limit presidents togo two te...	worldnews	true
3	seoul reuters former us presidential candidate...	politicsNews	true
4	washington reuters trump administration simple...	politicsNews	true
Basic data exploration
# How many articles per subject?
print(data.groupby(['subject'])['text'].count())
data.groupby(['subject'])['text'].count().plot(kind="bar")
plt.show()
subject
Government News     1570
Middle-east          778
News                9050
US_News              783
left-news           4459
politics            6841
politicsNews       11272
worldnews          10145
Name: text, dtype: int64

# How many fake and real articles?
print(data.groupby(['target'])['text'].count())
data.groupby(['target'])['text'].count().plot(kind="bar")
plt.show()
target
fake    23481
true    21417
Name: text, dtype: int64

# Word cloud for fake news
from wordcloud import WordCloud

fake_data = data[data["target"] == "fake"]
all_words = ' '.join([text for text in fake_data.text])

wordcloud = WordCloud(width= 800, height= 500,
                          max_font_size = 110,
                          collocations = False).generate(all_words)

plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Word cloud for real news
from wordcloud import WordCloud

real_data = data[data["target"] == "true"]
all_words = ' '.join([text for text in fake_data.text])

wordcloud = WordCloud(width= 800, height= 500,
                          max_font_size = 110,
                          collocations = False).generate(all_words)

plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Most frequent words counter (Code adapted from https://www.kaggle.com/rodolfoluna/fake-news-detector)   
from nltk import tokenize

token_space = tokenize.WhitespaceTokenizer()

def counter(text, column_text, quantity):
    all_words = ' '.join([text for text in text[column_text]])
    token_phrase = token_space.tokenize(all_words)
    frequency = nltk.FreqDist(token_phrase)
    df_frequency = pd.DataFrame({"Word": list(frequency.keys()),
                                   "Frequency": list(frequency.values())})
    df_frequency = df_frequency.nlargest(columns = "Frequency", n = quantity)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequency, x = "Word", y = "Frequency", color = 'blue')
    ax.set(ylabel = "Count")
    plt.xticks(rotation='vertical')
    plt.show()
# Most frequent words in fake news
counter(data[data["target"] == "fake"], "text", 20)

# Most frequent words in real news
counter(data[data["target"] == "true"], "text", 20)

Modeling
# Function to plot the confusion matrix (code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)
from sklearn import metrics
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
Peparing the data
# Split the data
X_train,X_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2, random_state=42)
Logistic regression
# Vectorizing and applying TF-IDF
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

# Fitting the model
model = pipe.fit(X_train, y_train)

# Accuracy
prediction = model.predict(X_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
accuracy: 98.76%
cm = metrics.confusion_matrix(y_test, prediction)
plot_confusion_matrix(cm, classes=['Fake', 'Real'])
Confusion matrix, without normalization

Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

# Vectorizing and applying TF-IDF
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', DecisionTreeClassifier(criterion= 'entropy',
                                           max_depth = 20, 
                                           splitter='best', 
                                           random_state=42))])
# Fitting the model
model = pipe.fit(X_train, y_train)

# Accuracy
prediction = model.predict(X_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
accuracy: 99.71%
cm = metrics.confusion_matrix(y_test, prediction)
plot_confusion_matrix(cm, classes=['Fake', 'Real'])
Confusion matrix, without normalization

Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', RandomForestClassifier(n_estimators=50, criterion="entropy"))])

model = pipe.fit(X_train, y_train)
prediction = model.predict(X_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
accuracy: 98.98%
cm = metrics.confusion_matrix(y_test, prediction)
plot_confusion_matrix(cm, classes=['Fake', 'Real'])
Confusion matrix, without normalization


SyntaxError: invalid syntax (<ipython-input-1-cdaabf834096>, line 11)